In [140]:
import numpy as np
import pandas as pd
import os
import seaborn as sns
import matplotlib.pyplot as plt
import Bio
import statsmodels.api as sm
import re
import ast

In [141]:
path: str = './data/'
prokaryotes: pd.DataFrame = pd.read_csv(os.path.join(path, "prokaryotes_sec_structure.csv"))

In [142]:
secs = ['Helix1','Turn1','Sheet1','Helix2','Sheet2']
for p in range(len(prokaryotes)):
    for s in secs:
        if pd.isnull(prokaryotes.at[p, s]) == False:    	
            prokaryotes.at[p, s] = ast.literal_eval(prokaryotes.at[p, s])

In [143]:
print(len(prokaryotes.loc[18,'Sheet1']))
print(len(prokaryotes.loc[18,'Sheet2']))

40
33


In [144]:
secs1 = ['Helix1','Turn1','Sheet1']
secs2 = ['Helix2','Sheet2']

Adding columns for total number of helices, sheets and coils

In [145]:
for u in range(len(prokaryotes)):
    for s in secs1:
        if isinstance(prokaryotes.at[u, s], list):
            prokaryotes.loc[u,f'{s}count'] = len(prokaryotes.at[u, s])
        else: 
            prokaryotes.loc[u,f'{s}count'] = np.nan
for u in range(len(prokaryotes)):
    for s in secs2:
        if isinstance(prokaryotes.at[u, s], list):
            prokaryotes.loc[u,f'{s}count'] = len(prokaryotes.at[u, s])
        else: 
            prokaryotes.loc[u,f'{s}count'] = np.nan

Adding columns for relative Helix and Sheet abundance

In [146]:
for u in range(len(prokaryotes)):
    for s in secs1:
        if isinstance(prokaryotes.at[u, s], list):
            sumel = 0
            for sl in prokaryotes.at[u, s]:
                sumel += len(sl)
            prokaryotes.loc[u,f'{s}perc'] = sumel/prokaryotes.loc[u,'Length']
        else: 
            prokaryotes.loc[u,f'{s}perc'] = np.nan
for u in range(len(prokaryotes)):
    for s in secs2:
        if isinstance(prokaryotes.at[u, s], list):
            sumel = 0
            for sl in prokaryotes.at[u, s]:
                sumel += len(sl)
            prokaryotes.loc[u,f'{s}perc'] = sumel/prokaryotes.loc[u,'Length']
        else: 
            prokaryotes.loc[u,f'{s}percs'] = np.nan

Adding columns for average Helix and Sheet length

In [147]:
for u in range(len(prokaryotes)):
    for s in secs1:
        if isinstance(prokaryotes.at[u, s], list):
            sumel = 0
            for sl in prokaryotes.at[u, s]:
                sumel += len(sl)
            prokaryotes.loc[u,f'{s}avg'] = sumel/prokaryotes.loc[u,f'{s}count']
        else: 
            prokaryotes.loc[u,f'{s}avg'] = np.nan
for u in range(len(prokaryotes)):
    for s in secs2:
        if isinstance(prokaryotes.at[u, s], list):
            sumel = 0
            for sl in prokaryotes.at[u, s]:
                sumel += len(sl)
            prokaryotes.loc[u,f'{s}avg'] = sumel/prokaryotes.loc[u,f'{s}count']
        else: 
            prokaryotes.loc[u,f'{s}avg'] = np.nan

C:\Users\tobia\AppData\Local\Temp\ipykernel_23884\2786584450.py:16: RuntimeWarning: invalid value encountered in scalar divide
  prokaryotes.loc[u,f'{s}avg'] = sumel/prokaryotes.loc[u,f'{s}count']
C:\Users\tobia\AppData\Local\Temp\ipykernel_23884\2786584450.py:16: RuntimeWarning: invalid value encountered in scalar divide
  prokaryotes.loc[u,f'{s}avg'] = sumel/prokaryotes.loc[u,f'{s}count']
C:\Users\tobia\AppData\Local\Temp\ipykernel_23884\2786584450.py:16: RuntimeWarning: invalid value encountered in scalar divide
  prokaryotes.loc[u,f'{s}avg'] = sumel/prokaryotes.loc[u,f'{s}count']
C:\Users\tobia\AppData\Local\Temp\ipykernel_23884\2786584450.py:16: RuntimeWarning: invalid value encountered in scalar divide
  prokaryotes.loc[u,f'{s}avg'] = sumel/prokaryotes.loc[u,f'{s}count']
C:\Users\tobia\AppData\Local\Temp\ipykernel_23884\2786584450.py:16: RuntimeWarning: invalid value encountered in scalar divide
  prokaryotes.loc[u,f'{s}avg'] = sumel/prokaryotes.loc[u,f'{s}count']
C:\Users\tobia\

Adding column for relative fraction of secondary structures (Helix and Beta sheet combined)

In [148]:
for u in range(len(prokaryotes)):
    for s in secs1:
        if isinstance(prokaryotes.at[u, s], list):
            prokaryotes.loc[u,'secstr1'] = prokaryotes.loc[u,'Helix1perc']+prokaryotes.loc[u,'Turn1perc']+prokaryotes.loc[u,'Sheet1perc']
for u in range(len(prokaryotes)):
    for s in secs2:
        if isinstance(prokaryotes.at[u, s], list):
            prokaryotes.loc[u,'secstr2'] = prokaryotes.loc[u,'Helix2perc']+prokaryotes.loc[u,'Sheet2perc']

In [149]:
prokaryotes_temp_m = prokaryotes.sort_values('meltPoint',ascending = False,na_position="first")
prokaryotes_temp_NaN = prokaryotes_temp_m.dropna(subset='meltPoint')
prokaryotes_temp_NaN['meltPoint'] = prokaryotes_temp_NaN['meltPoint'].astype(float)
prokaryotes_temp_NaN['temperature'] = prokaryotes_temp_NaN['temperature'].astype(float)
prokaryotes_temp_NaN['auc'] = prokaryotes_temp_NaN['auc'].astype(float)

quantiles = prokaryotes_temp_NaN['meltPoint'].quantile(q=[0.1, 0.9])
quantiles = quantiles.astype('float64')
quantiles = list(quantiles)

pain = []
quantiles_prokaryotes_temp_NaN = pd.DataFrame()
for n in range(len(prokaryotes_temp_NaN)):
    if prokaryotes_temp_NaN.iloc[n,11] < quantiles[0] or prokaryotes_temp_NaN.iloc[n, 11] > quantiles[1]:
        pain.append(n)
quantiles_prokaryotes_temp_NaN = prokaryotes_temp_NaN.iloc[pain,:]
quantiles_prokaryotes_temp_NaN.reset_index(drop=True, inplace=True)

C:\Users\tobia\AppData\Local\Temp\ipykernel_23884\332300994.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  prokaryotes_temp_NaN['meltPoint'] = prokaryotes_temp_NaN['meltPoint'].astype(float)
C:\Users\tobia\AppData\Local\Temp\ipykernel_23884\332300994.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  prokaryotes_temp_NaN['temperature'] = prokaryotes_temp_NaN['temperature'].astype(float)
C:\Users\tobia\AppData\Local\Temp\ipykernel_23884\332300994.py:5: SettingWithCopyWarning: 
A value is trying to be s

In [97]:
quantiles_prokaryotes_temp_NaN.corr(method='spearman',numeric_only=True)['meltPoint']

Length        -0.076928
temperature    0.828756
fold_change    0.084312
meltPoint      1.000000
auc            0.907914
Helix1count    0.307083
Turn1count     0.026546
Sheet1count    0.210829
Helix2count   -0.080430
Sheet2count    0.053873
Helix1perc     0.105795
Turn1perc     -0.151912
Sheet1perc     0.111733
Helix2perc    -0.111735
Sheet2perc     0.130885
Helix1avg     -0.154751
Turn1avg       0.020562
Sheet1avg     -0.020224
Helix2avg     -0.095630
Sheet2avg     -0.017054
secstr1        0.238029
secstr2       -0.070365
Name: meltPoint, dtype: float64

In [115]:
aacids = ['A', 'V', 'I', 'L', 'M', 'F', 'W','N', 'Q', 'S', 'T', 'Y','D', 'E','R', 'H', 'K', 'C', 'P', 'G' ]
from function import rel_aa_comp
aagl = []
for n in aacids:
    for m in aacids:
        if n != m and [m,n] not in aagl:
            aagl.append([n,m])

In [158]:
for g in aagl:
    for v in range(len(prokaryotes)):
        prokaryotes.loc[v,f'{g[0]}{g[1]}'] = rel_aa_comp(prokaryotes.loc[v,'Sequence'],g)
    if prokaryotes[f'{g[0]}{g[1]}'].corr(prokaryotes['meltPoint'],method = 'pearson') < 0.35 and prokaryotes[f'{g[0]}{g[1]}'].corr(prokaryotes['meltPoint'],method = 'pearson') > -0.35:
        prokaryotes.drop(columns=[f'{g[0]}{g[1]}'],inplace=True)

C:\Users\tobia\AppData\Local\Temp\ipykernel_23884\4108521563.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  prokaryotes.loc[v,f'{g[0]}{g[1]}'] = rel_aa_comp(prokaryotes.loc[v,'Sequence'],g)
C:\Users\tobia\AppData\Local\Temp\ipykernel_23884\4108521563.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  prokaryotes.loc[v,f'{g[0]}{g[1]}'] = rel_aa_comp(prokaryotes.loc[v,'Sequence'],g)
C:\Users\tobia\AppData\Local\Temp\ipykernel_23884\4108521563.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the resu

KeyboardInterrupt: 

In [157]:
correlation = prokaryotes.corr(method='spearman',numeric_only=True)

Geobacillus stearothermophilus NCA26 lysate


Calculating amino acid percentage in (helices and sheets)

In [168]:
import itertools
test = [[1,2,3],[4,5,6],[7,8,9]]
tests = str(list(itertools.chain.from_iterable(test)))
print(type(tests))

<class 'str'>


In [189]:
for a in aacids:
    for y in range(len(prokaryotes)):
        helixind = list(itertools.chain.from_iterable(prokaryotes.loc[y,'Helix2']))
        helixseq = ''
        for u in helixind:
            if u < len(prokaryotes.loc[y,'Sequence']):
                helixseq += prokaryotes.loc[y,'Sequence'][u]
        if len(prokaryotes.loc[y,'Helix2']) != 0:
            prokaryotes.loc[y,f'{a}helixcomp'] = rel_aa_comp(helixseq,a)

C:\Users\tobia\AppData\Local\Temp\ipykernel_23884\390532804.py:9: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  prokaryotes.loc[y,f'{a}helixcomp'] = rel_aa_comp(helixseq,a)
C:\Users\tobia\AppData\Local\Temp\ipykernel_23884\390532804.py:9: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  prokaryotes.loc[y,f'{a}helixcomp'] = rel_aa_comp(helixseq,a)
C:\Users\tobia\AppData\Local\Temp\ipykernel_23884\390532804.py:9: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, whic

In [194]:
b = prokaryotes.corr(method='pearson',numeric_only=True)['meltPoint']

In [193]:
print(prokaryotes.loc[12,'Sequence'][951])

E
